# Experimentos: Baseline com Random Forest

Este notebook estabelece baselines de performance com um modelo de Random Forest clássico em datasets

# Dataset = Breast Cancer Wisconsin (Diagnostic)

In [9]:
from sklearn.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier  # Importando Random Forest
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

## Loading data and initial analysis

In [10]:
X, y = load_breast_cancer(return_X_y=True)  # Carregando o dataset

# Verificando as dimensões dos nossos dados
print(f"Dimensões dos dados: {X.shape}")


# Vendo a distribuição das classes (0 e 1)
print("\nDistribuição das classes:")
# Usando numpy para contar as ocorrências de cada classe
unique, counts = np.unique(y, return_counts=True)
for class_label, count in zip(unique, counts):
    print(f"Classe {class_label}: {count} ocorrências")


Dimensões dos dados: (569, 30)

Distribuição das classes:
Classe 0: 212 ocorrências
Classe 1: 357 ocorrências


## Split data into training and testing sets

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y) # Stratify -> mantém a proporção das classes no split

print(f"Dimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")


Dimensões do conjunto de treino: (426, 30)
Dimensões do conjunto de teste: (143, 30)


## Normalize data

In [12]:
# normalizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Ajusta o scaler e transforma os dados de treino
X_test = scaler.transform(X_test)  # Transforma os dados de teste com o mesmo scaler

print("\nDados normalizados")






Dados normalizados


# Training Random Forest model

In [13]:
# Treinando o modelo Random Forest
rf_model_bc = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_bc.fit(X_train, y_train)

print("\nModelo treinado com Random Forest")


Modelo treinado com Random Forest


# Predictions

In [14]:
# predicting the test set com Random Forest
rf_predictions_bc = rf_model_bc.predict(X_test)
# Avaliando o modelo
print("\nRelatório de Classificação:")
print(classification_report(y_test, rf_predictions_bc))


Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        53
           1       0.96      0.98      0.97        90

    accuracy                           0.96       143
   macro avg       0.96      0.95      0.95       143
weighted avg       0.96      0.96      0.96       143



# Dataset = Student Performance

In [15]:
from imblearn.over_sampling import SMOTE


def run_rf_all_features(filepath, subject_name, separator=','):

    print(f"   Iniciando processamento para Random Forest no dataset de ({subject_name})    ")
    
    # Carregamento e Pré-processamento 
    df = pd.read_csv(filepath, sep=separator)
    y_grades = df[['G1', 'G2', 'G3']]
    X_features = df.drop(columns=['G1', 'G2', 'G3'])
    y_final = np.where(y_grades['G3'] >= 10, 1, 0)
    X_final = pd.get_dummies(X_features, drop_first=True)

    # Divisão, Normalização e SMOTE
    X_train, X_test, y_train, y_test = train_test_split(
        X_final, y_final, test_size=0.25, random_state=42, stratify=y_final
    )
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
    
    # Treinamento e Avaliação do Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train_resampled, y_train_resampled)
    
    predictions = rf_model.predict(X_test_scaled)
    
    print(f"\n   Relatório de Classificação    ")
    print(classification_report(y_test, predictions, target_names=['Reprovado', 'Aprovado']))

# Execution

In [16]:

path_portugues = '../data/student-por.csv'
path_matematica = '../data/student-mat.csv'


run_rf_all_features(filepath=path_portugues, subject_name="Português", separator=",")

print("\n" + "="*50 + "\n") 


run_rf_all_features(filepath=path_matematica, subject_name="Math", separator=";")

   Iniciando processamento para Random Forest no dataset de (Português)    

   Relatório de Classificação    
              precision    recall  f1-score   support

   Reprovado       0.32      0.28      0.30        25
    Aprovado       0.87      0.89      0.88       138

    accuracy                           0.80       163
   macro avg       0.60      0.59      0.59       163
weighted avg       0.79      0.80      0.79       163



   Iniciando processamento para Random Forest no dataset de (Math)    

   Relatório de Classificação    
              precision    recall  f1-score   support

   Reprovado       0.52      0.33      0.41        33
    Aprovado       0.72      0.85      0.78        66

    accuracy                           0.68        99
   macro avg       0.62      0.59      0.59        99
weighted avg       0.65      0.68      0.65        99

